<a href="https://colab.research.google.com/github/LEECHANGDAEE/colab/blob/main/103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from re import X
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
import warnings

warnings.filterwarnings(action='ignore') 

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, get_scorer
from sklearn.linear_model import Lasso, Ridge, LassoCV,LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split
from numpy import arange
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [ ]:
train = pd.read_csv('./train2.csv',encoding='cp949')
test = pd.read_csv('./test1.csv',encoding='cp949')
submission = pd.read_csv('./sample_submission1.csv',encoding='cp949')

In [ ]:
train.groupby('지역').mean()

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
지역,,,,,,,,
강원도,549.288770,45.123209,95.000000,8.989305,0.000000,3.422460,366.679144,388.245989
경기도,963.542857,44.284319,115.613445,12.831933,0.210084,4.363025,822.151261,797.400000
경상남도,698.206612,41.752645,97.000000,11.498623,0.000000,3.834711,465.983471,409.685950
경상북도,610.693694,45.261892,108.810811,21.117117,0.000000,3.018018,571.315315,613.855856
광주광역시,776.368794,43.744184,122.028369,17.063830,0.000000,3.496454,673.531915,731.858156
대구광역시,862.862745,47.854608,142.137255,17.862745,0.460784,3.735294,801.754902,873.519608
대전광역시,1285.838346,47.281090,95.409774,11.439850,0.722543,4.714286,693.233083,535.394737
부산광역시,1511.070122,47.214390,87.954268,13.689024,0.533537,2.448171,467.713415,316.545732
서울특별시,797.520000,49.783000,80.680000,9.760000,0.240000,4.400000,664.060000,682.780000


In [ ]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [ ]:
# local_map = {}
# for i, loc in enumerate(train['지역'].unique()):
#     local_map[loc] = i

# train['지역'] = train['지역'].map(local_map)
# test['지역'] = test['지역'].map(local_map)

train['전용면적'] = round(train['전용면적'])//5*5
test['전용면적'] = round(test['전용면적'])//5*5

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 1
idx = train[train['전용면적']>50].index
train.loc[idx, '전용면적'] = 2
idx = train[train['전용면적']==50].index
train.loc[idx, '전용면적'] = 3
idx = train[train['전용면적']>40].index
train.loc[idx, '전용면적'] = 4
idx = train[train['전용면적']>9].index
train.loc[idx, '전용면적'] = 5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 1
idx = test[test['전용면적']>50].index
test.loc[idx, '전용면적'] = 2
idx = test[test['전용면적']==50].index
test.loc[idx, '전용면적'] = 3
idx = test[test['전용면적']>40].index
test.loc[idx, '전용면적'] = 4
idx = test[test['전용면적']>9].index
test.loc[idx, '전용면적'] = 5

train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

train_apt = train[train['임대건물구분']=='아파트']
train_st = train[train['임대건물구분']=='상가']
test_apt = test[test['임대건물구분']=='아파트']
test_st = test[test['임대건물구분']=='상가']

columns = ['단지코드', '공급유형', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []

In [ ]:
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train_apt['단지코드'].unique())):
    temp = train_apt[train_apt['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test_apt['단지코드'].unique())):
    temp = test_apt[test_apt['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

new_train = new_train.fillna(0)
new_test = new_test.fillna(0)

train_x = new_train.iloc[:,1:-1]
train_y = new_train.iloc[:,-1]

new_train2 = pd.get_dummies(train_x, columns=['공급유형'])
new_train2 = new_train2.fillna(0)
new_test2 = pd.get_dummies(new_test, columns=['공급유형'])
new_test2 = new_test2.fillna(0)

411it [00:04, 89.07it/s]
150it [00:01, 92.89it/s]


In [ ]:
for i in range(len(new_train2)) : 
  if new_train2.지역[i] in ['서울특별시','부산광역시'] :
    new_train2.지역[i] = '경부'

In [ ]:
for i in range(len(new_test2)) : 
  if new_test2.지역[i] in ['서울특별시','부산광역시'] :
   new_test2.지역[i] = '경부'

In [ ]:
# for i in range(len(new_train2)) :
#   if new_train2.지역[i] in ['충청남도','충청북도'] :
#     new_train2.지역[i] = '충청도'
#   elif new_train2.지역[i] in ['전라북도','전라남도'] :
#     new_train2.지역[i] = '전라도'
#   elif new_train2.지역[i] in ['경상남도','경상북도'] :
#     new_train2.지역[i] = '경상도'
new_train2 = pd.get_dummies(data = new_train2, columns = ['지역'], prefix = '지역')

In [ ]:
# for i in range(len(new_test2)) :
#   if new_test2.지역[i] in ['충청남도','충청북도'] :
#     new_test2.지역[i] = '충청도'
#   elif new_test2.지역[i] in ['전라북도','전라남도'] :
#     new_test2.지역[i] = '전라도'
#   elif new_test2.지역[i] in ['경상남도','경상북도'] :
#     new_test2.지역[i] = '경상도'
new_test2 = pd.get_dummies(data = new_test2, columns = ['지역'], prefix = '지역')

In [ ]:
# new_train2.총세대수 = np.sqrt(new_train2.총세대수)
new_train2.단지내주차면수 = np.sqrt(new_train2.단지내주차면수)

In [ ]:
# new_test2.총세대수 = np.sqrt(new_test2.총세대수)
new_test2.단지내주차면수 = np.sqrt(new_test2.단지내주차면수)

In [ ]:
# train_y = np.sqrt(train_y)

In [ ]:
new_test2 = new_test2.iloc[:,1:]
new_test2

,총세대수,공가수,단지내주차면수,지하철,버스,면적_5.0,면적_4.0,면적_3.0,면적_2.0,면적_1.0,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_행복주택,지역_강원도,지역_경기도,지역_경부,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도
0,754.0,14.0,26.134269,0.0,2.0,116.0,262.0,376.0,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1354.0,9.0,34.871192,0.0,3.0,420.0,356.0,578.0,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,619.0,18.0,23.388031,0.0,16.0,247.0,196.0,132.0,44.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,593.0,7.0,23.302360,0.0,3.0,335.0,174.0,84.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,1297.0,11.0,33.346664,0.0,2.0,425.0,568.0,304.0,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,349.0,17.0,16.431677,0.0,4.0,250.0,96.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
146,596.0,35.0,24.351591,0.0,1.0,484.0,107.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
147,120.0,9.0,6.324555,0.0,1.0,120.0,0.0,0.0,0.0,0.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148,675.0,38.0,21.610183,0.0,1.0,490.0,180.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
######################## train 데이터 내에서 실험하는 코드 ########################
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = train_test_split(new_train2, train_y, train_size=0.8, test_size=0.2, random_state =0)

# mlr = LinearRegression().fit(x_train, y_train)
# ridge = Ridge(alpha = 0.5).fit(x_train, y_train)
# lasso = Lasso(alpha=0.2).fit(x_train, y_train)

# y_linear_predict = mlr.predict(x_test)
# y_ridge_predict = ridge.predict(x_test)
# y_lasso_predict = lasso.predict(x_test)

# print('linear regression rmse : {}'.format(mean_squared_error(y_linear_predict,y_test)**0.5))
# print('ridge regression rmse : {}'.format(mean_squared_error(y_ridge_predict,y_test)**0.5))
# print('lasso regression rmse : {}'.format(mean_squared_error(y_lasso_predict,y_test)**0.5))

# print(mean_absolute_error(y_lasso_predict,y_test))

################################################################################

In [ ]:
x_train

,총세대수,공가수,단지내주차면수,지하철,버스,면적_5.0,면적_4.0,면적_3.0,면적_2.0,면적_1.0,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_행복주택,지역_강원도,지역_경기도,지역_경부,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도
326,396.0,8.0,15.394804,0.0,2.0,336.0,60.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
410,239.0,7.0,12.884099,0.0,1.0,0.0,38.0,182.0,19.0,0.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
370,890.0,20.0,20.566964,1.0,8.0,756.0,134.0,0.0,0.0,0.0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
206,646.0,4.0,24.145393,0.0,3.0,433.0,213.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
75,775.0,10.0,27.166155,0.0,4.0,465.0,0.0,310.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1546.0,1.0,34.292856,1.0,2.0,1262.0,284.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
192,757.0,21.0,27.586228,0.0,12.0,0.0,330.0,48.0,379.0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
117,410.0,16.0,20.049938,0.0,20.0,223.0,187.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
47,840.0,7.0,24.372115,1.0,1.0,0.0,0.0,674.0,166.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
random_grid = {'alpha':  [round(x,1) for x in arange(0, 1, 0.01)]}
from sklearn.model_selection import RandomizedSearchCV,cross_val_score, KFold
kfold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

lasso = Lasso()
# lasso_random = RandomizedSearchCV(estimator = lasso, param_distributions = random_grid, n_iter = 100, cv = kfold, verbose=2, n_jobs = -1)
lasso_random = GridSearchCV(estimator= lasso, param_grid= random_grid, scoring='neg_mean_absolute_error', cv=kfold, n_jobs=-1)
lasso_random_cv = lasso_random.fit(x_train,y_train)
lasso_random_cv.best_estimator_
lasso_optimal = lasso_random_cv.best_estimator_.fit(x_train,y_train)

In [ ]:
la_op_pred = lasso_optimal.predict(x_test)
print('lasso regression rmse : {}'.format(mean_squared_error(la_op_pred,y_test)**0.5))
print(mean_absolute_error(la_op_pred,y_test))

lasso regression rmse : 131.1592199461499
90.5563997260903


In [ ]:
print('훈련점수 :',lasso_optimal.score(x_train, y_train))
print('테스트 점수 :',lasso_optimal.score(x_test, y_test))
print('사용한 특성 수 :', np.sum(lasso_optimal.coef_ != 0))

훈련점수 : 0.822169938736725
테스트 점수 : 0.8761353516339659
사용한 특성 수 : 24


In [ ]:
lasso_optimal = lasso_random_cv.best_estimator_.fit(new_train2,train_y)

In [ ]:
y_lasso_predict = lasso_optimal.predict(new_test2)
y_lasso_predict = y_lasso_predict
submission['num'] = y_lasso_predict

submission.to_csv('lasso.csv', index=False)

In [ ]:
# y_lasso_predict[147] = 15 # 자꾸 마이너스 나와서 일단 넣어둠
# print(y_lasso_predict)

# submission['num'] = y_lasso_predict

# submission.to_csv('test.csv', index=False)